In [102]:
%cd ..

/home/ec2-user/SageMaker


In [103]:
!rm -rf Sloveniinf

In [1]:
%%capture

#enable the process bar
!conda install -y -c conda-forge ipywidgets
!jupyter nbextension enable --py widgetsnbextension

!conda install -y pytorch torchaudio cudatoolkit=10.2 -c pytorch
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer
!pip install s3fs
!pip install boto3

print('everything installed')

In [1]:
# %%capture
# !pip install datasets==1.13.3
# !pip install transformers==4.11.3

UsageError: %%capture is a cell magic, but the cell body is empty.


In [2]:
import boto3
import json
import numpy as np
import librosa

In [3]:
from datasets import load_dataset, load_metric, Audio


In [6]:
# import keys from  local 


In [7]:
s3resource = boto3.resource(
service_name='s3',
region_name='eu-west-1',
aws_access_key_id= data['aws_access_key_id'],
aws_secret_access_key= data['aws_secret_access_key'] )

In [ ]:
# s3client = boto3.client(
# service_name='s3',
# region_name='eu-west-1',
# aws_access_key_id= data['aws_access_key_id'],
# aws_secret_access_key= data['aws_secret_access_key'] )

In [8]:
lang ='Sloveni'
BUCKET = 'commonvoicesdataset'
FILE_TO_READ = '{}/vocab.json'.format(lang)
object1 = s3resource.Object(BUCKET,FILE_TO_READ)
file_content = object1.get()['Body'].read().decode('utf-8')
A =json.loads(file_content)

In [9]:
import json
with open('./vocab.json', 'w') as vocab_file:
    json.dump(A, vocab_file)

In [10]:
from datasets import load_from_disk

In [66]:
from datasets.filesystems import S3FileSystem
s3fsystems = S3FileSystem(key=data['aws_access_key_id'], secret=data['aws_secret_access_key'])  

In [67]:
train_set_loaded = load_from_disk('s3://commonvoicesdataset/Sloveni/train_set',fs=s3fsystems)  

In [68]:
test_set_loaded = load_from_disk('s3://commonvoicesdataset/Sloveni/test_set',fs=s3fsystems)  

In [65]:
test_set_loaded

NameError: name 'test_set_loaded' is not defined

In [98]:
common_voice_test_transcription = load_dataset("common_voice", "tr", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration tr-ad9f7b76efa9f3a0
Reusing dataset common_voice (/home/ec2-user/.cache/huggingface/datasets/common_voice/tr-ad9f7b76efa9f3a0/6.1.0/f7a9d973839b7706e9e281c19b7e512f31badf3c0fdbd21c671f3c4bf9acf3b9)


In [11]:
!aws s3 cp   s3://commonvoicesdataset/Sloveni/finalmodel  '/home/ec2-user/SageMaker/Sloveniinf' --recursive 

download: s3://commonvoicesdataset/Sloveni/finalmodel/config.json to ../Sloveniinf/config.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/special_tokens_map.json to ../Sloveniinf/special_tokens_map.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/vocab.json to ../Sloveniinf/vocab.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/training_args.bin to ../Sloveniinf/training_args.bin
download: s3://commonvoicesdataset/Sloveni/finalmodel/preprocessor_config.json to ../Sloveniinf/preprocessor_config.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/tokenizer_config.json to ../Sloveniinf/tokenizer_config.json
download: s3://commonvoicesdataset/Sloveni/finalmodel/pytorch_model.bin to ../Sloveniinf/pytorch_model.bin


In [12]:
path_to_model =  '/home/ec2-user/SageMaker/Sloveniinf'

In [13]:
%%capture
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(path_to_model).to("cuda")


In [14]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor.from_pretrained(path_to_model)

In [17]:
test_set_loaded[0].keys()

dict_keys(['labels', 'input_values'])

In [15]:
import torch

In [70]:
test_set_loaded[1]["input_values"][:20]

[0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868,
 0.00021374753850977868]

In [99]:
with open('/home/ec2-user/SageMaker/Sloveniinf/results.text','w') as file:
    

    for sample_num in range(1,20,4):
        input_dict = processor(test_set_loaded[sample_num]["input_values"], return_tensors="pt", padding=True,sampling_rate=16000)

        logits = model(input_dict.input_values.to("cuda")).logits

        pred_ids = torch.argmax(logits, dim=-1)[0]
        
#         print("\nReference: ",common_voice_test_transcription[sample_num]["sentence"].lower())
    #   print(common_voice_test_transcription[sample_num]["sentence"].lower())
#         print("Prediction:",processor.decode(pred_ids))
    #   print(processor.decode(pred_ids))
#         print('------------------------------------------')
        text1 ="\nReference: ",common_voice_test_transcription[sample_num]["sentence"].lower()
        text2 = "\nPrediction:",processor.decode(pred_ids)
#         print('------------------------------------------')
#         file.write(text1)
#         file.write(text2)
        file.writelines(text1)
        file.writelines(text2)
        file.writelines('\n---------------------------------')

In [95]:
input_dict = processor(test_set_loaded[11]["input_values"], return_tensors="pt", padding=True,sampling_rate=16000)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

processor.decode(pred_ids)


''

In [96]:
test_set_loaded[11]["input_values"]

[0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,
 0.00016356426931452006,


In [61]:
y,s = librosa.load('snd.wav',sr=16000) # Downsample 44.1kHz to 8kHz

In [92]:
y[10000:10100]

array([-2.71540158e-03, -3.96161713e-03, -5.42724319e-03, -6.93964865e-03,
       -8.05987045e-03, -8.79814941e-03, -9.09232069e-03, -8.73983931e-03,
       -7.97839928e-03, -6.90991338e-03, -5.77865075e-03, -4.75672493e-03,
       -3.85415996e-03, -3.11989710e-03, -2.32558930e-03, -1.37559662e-03,
       -2.51180842e-04,  1.31890050e-03,  3.47804325e-03,  6.09425409e-03,
        9.02848970e-03,  1.20117627e-02,  1.47470348e-02,  1.68872345e-02,
        1.83952693e-02,  1.93772372e-02,  1.96368136e-02,  1.93978846e-02,
        1.89361982e-02,  1.83668602e-02,  1.77592542e-02,  1.71504356e-02,
        1.64907277e-02,  1.55159710e-02,  1.41407996e-02,  1.23954192e-02,
        1.03290686e-02,  8.04610085e-03,  5.48654050e-03,  2.71501951e-03,
        7.66258818e-05, -2.37136241e-03, -4.62705269e-03, -6.79183472e-03,
       -8.83183442e-03, -1.03911338e-02, -1.15345698e-02, -1.22317700e-02,
       -1.23172123e-02, -1.20641971e-02, -1.13124605e-02, -1.00850286e-02,
       -9.06740781e-03, -

In [86]:
# input_dict = processor(test_set_loaded[sample_num]["input_values"], return_tensors="pt", padding=True,sampling_rate=16000)
input_dict  = processor(np.array(y),return_tensors = "pt",padding = True,sampling_rate = 16000)
input_dict.input_values *-1

tensor([[ 5.1643e-05,  5.1643e-05,  5.1643e-05,  ..., -9.2531e-05,
          4.9660e-04, -2.6118e-04]])

In [81]:
list(input_dict.input_values-1)[:10]


[tensor([-0.0005, -0.0005, -0.0005,  ...,  0.0009, -0.0050,  0.0026])]

In [87]:


logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]
        
# text1 ="\nReference: ",common_voice_test_transcription[sample_num]["sentence"].lower()
t=processor.decode(pred_ids)


In [88]:
pred_ids

tensor([33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33], device='cuda:0')

In [1]:
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=577a2926db83d1ab7822bfff1dabd169f5975720bce2b2d901d4411b88cec048
  Stored in directory: /home/ec2-user/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [ ]:
from os import path
from pydub import AudioSegment

# files                                                                         
src = "transcript.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")